# Creating a Cutout with ERA5

In this example we download ERA5 data on-demand for a cutout we want to create.
(atlite does also work with other datasources, but ERA5 is the easiest one to get started.)

This only works if you have in before

* Installed the Copernicus Climate Data Store `cdsapi` package
* Registered and setup your CDS API key as described [on their website here](https://cds.climate.copernicus.eu/api-how-to)

Import the package first:

In [ ]:
import atlite

We implement notifications in `atlite` using loggers from the `logging` library.

We recommend you always launch a logger to get information on what is going on.
For debugging, you can use the more verbose `level=logging.DEBUG`:

In [ ]:
import logging

logging.basicConfig(level=logging.INFO)

## Defining the Cutout extent

> This will not yet trigger any major operations.

A cutout is the basis for any of your work and calculations.

The `cutout` is created in the directory and file specified by the relative `path`
If a cutout at the given location already exists, then this command will simply load the cutout again.
If the cutout does not yet exist, it will specify the new to-be-created cutout.
> **Note** `ERA5`, Before the data can be downloaded it has to be processed by CDS servers, this might take a while depending on the ammout of data requested. 
You can check the status of your request [here](https://cds.climate.copernicus.eu/cdsapp#!/yourrequests).

In [ ]:
cutout = atlite.Cutout(
    path="western-europe-2011-01.nc",
    module="era5",
    x=slice(-13.6913, 1.7712),
    y=slice(49.9096, 60.8479),
    time="2011-01",
)

For creating the cutout, you need to specify

* The dataset to create the cutout with (`era5`)
* The time period it covers
* The longitude `x` and latitude `y` it stretches


Here we went with the `ERA5` dataset from ECMWF

```
module="era5"
```

Here we decided to provide the `time` period of the cutout as a string, because it is only a month.
You could have also specify it as a time range

```
slice("2011-01","2011-01")
```

The regional bounds (space the cutout stretches) where specified by the
```
x=slice(-13.6913, 1.7712) # Longitude
y=slice(49.9096, 60.8479) # Latitude
```

and describe a rectangle's edges.
In this case we drew a rectangle containing some parts of the atlantic ocean,
the Republic of Ireland and the UK.

## Preparing the Cutout

If the cutout does not yet exist or has some features which are not yet included, we have to tell atlite to go ahead and do so.

No matter which dataset you use, this is where all the work actually happens.
This can be fast or take some or a lot of time and resources, among others depending on
your computer ressources and (for downloading e.g. ERA5 data) your internet connection.

In [ ]:
cutout.prepare()

The `cutout.prepare()` function takes a list of features which should be prepared. When this is not specified, all available features are build. 

After, the execution all downloaded data is stored at `cutout.path`. Per default it is not loaded into memory, but into [dask](https://dask.org/) arrays. This keeps the memory consumption extremely low.  

The data is accessible in `cutout.data` which is an `xarray.Dataset`. 
Querying the cutout gives us some basic information on which data is contained
in it.

In [ ]:
cutout.data

We can again breakdown which data array belongs to which feature.

In [ ]:
cutout.prepared_features

If you have matplotlib installed, you can directly use the 
plotting functionality from `xarray` to plot features from
the cutout's data.

> **Warning**
>  This will trigger `xarray` to load all the corresponding data from disk into memory!



Now that your cutout is created and prepared, you can call conversion functions as `cutout.pv` or `cutout.wind`. Note that this requires a bit more information, like what kind of pv panels to use, where do they stand etc. Please have a look at the other examples to get a picture of application cases.

## Reducing Cutout file sizes

Cutouts can become quite large, depending on the spatial and temporal scope they cover.
By default `atlite` uses a trade-off between speed and compression to reduce the file size of cutouts.

Stronger compression can be selected when creating a new cutout by choosing a higher `complevel` (`1` to `9`, default: `4`)
```
cutout.prepare(compression={"zlib": True, "complevel": 9})
```

To change the compression for an existing cutout:
```
cutout = atlite.Cutout("cutout-path.nc")

compression = {"zlib": True, "complevel": 9}
for var in cutout.data.data_vars:
    cutout.data[var].encoding.update(compression)

cutout.to_file()
```
For details and more arguments for `compression`, see the [xarray documentation](https://docs.xarray.dev/en/stable/generated/xarray.Dataset.to_netcdf.html) for details.

Alternatively a cutout can also be compressed by using the `netcdf` utility `nccopy` from the commandline:

```
nccopy -d4 -s <input cutout .nc file> <output cutout .nc file>
```